In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

In [ ]:
# !pip install transformers==2.1.0
!pip install gensim==4.1.2
!pip3 install --user underthesea

# !pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from pyvi import ViTokenizer, ViPosTagger
import regex as re
from underthesea import word_tokenize
import numpy as np
import pandas as pd
import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
#from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Thống kê số lượng data theo nhãn
count = {}
for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    key = line.split()[0]
    count[key] = count.get(key, 0) + 1

for key in count:
    print(key, count[key])

__label__thể_thao 2611
__label__âm_nhạc 2595
__label__nhịp_sống 2613
__label__công_nghệ 2595
__label__thời_sự 2607
__label__thế_giới 2602
__label__thời_trang 2596
__label__du_lịch 2593
__label__sống_trẻ 2602
__label__giáo_dục 2603
__label__kinh_doanh 2597
__label__pháp_luật 2592
__label__giải_trí 2604
__label__phim_ảnh 2596
__label__xe_360 2602
__label__ẩm_thực 2482
__label__xuất_bản 2599
__label__sức_khỏe 2589


In [ ]:
# Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 18
vocab = {}
label_vocab = {}
# for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
#     words = line.split()
#     # lưu ý từ đầu tiên là nhãn
#     label = words[0]
#     if label not in label_vocab:
#         label_vocab[label] = {}
#     for word in words[1:]:
#         label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
#         if word not in vocab:
#             vocab[word] = set()
#         vocab[word].add(label)

# count = {}
# for word in vocab:
#     if len(vocab[word]) == total_label:
#         count[word] = min([label_vocab[x][word] for x in label_vocab])
        
# sorted_count = sorted(count, key=count.get, reverse=True)
# for word in sorted_count[:100]:
#     print(word, count[word])
text = []
label = []

for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    words = line.strip().split()
    label.append(words[0])
    text.append(' '.join(words[1:]))

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(label)
encode_label = label_encoder.transform(label)

In [ ]:
import pandas as pd
df = pd.DataFrame(text, columns=["text"])
df.to_csv('text.csv', index=False)

In [ ]:
rd = pd.read_csv('text.csv')
rd[:10]

,text
0,sanchez không dự trận khai màn nh anh cùng ars...
1,những sao việt hóa_thân thành phó nháy trong m...
2,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
import pandas as pd
df = pd.DataFrame(encode_label, columns=["label"])
df.to_csv('label.csv', index=False)

In [ ]:
lb = pd.read_csv('label.csv')
lb[:10]

,label
0,11
1,16
2,5
3,0
4,11
5,12
6,10
7,13
8,1
9,12


In [ ]:
a = pd.read_csv("label.csv")
b = pd.read_csv("text.csv")
df_merge_col = pd.merge(a, b,left_index=True, right_index=True, how='outer')
df_merge_col.to_csv("data.csv", index=False)

In [ ]:
c = pd.read_csv("data.csv")
c[:10]

,label,text
0,11,sanchez không dự trận khai màn nh anh cùng ars...
1,16,những sao việt hóa_thân thành phó nháy trong m...
2,5,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,0,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,11,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,12,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,10,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,13,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,1,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,12,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
df = c.copy()
df[:10]

,label,text
0,11,sanchez không dự trận khai màn nh anh cùng ars...
1,16,những sao việt hóa_thân thành phó nháy trong m...
2,5,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,0,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,11,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,12,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,10,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,13,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,1,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,12,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
df.shape

(46678, 2)

In [ ]:
#tok = Tokenizer()
# lst_corpus = []
# for string in df['text']:
#    lst_words = string.split()
#    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, len(lst_words), 1)]
#    lst_corpus.append(lst_grams)

# print(len(lst_corpus))


In [ ]:
split_text = [s.split() for s in text]

In [ ]:
w2v = gensim.models.Word2Vec(split_text,
                                   vector_size=100,
                                   window=5,
                                   min_count=1)
words = set(w2v.wv.index_to_key)

In [ ]:
w2v.train(split_text ,epochs=10,total_examples=len(split_text ))

(227654115, 256614670)

In [ ]:
vocab=w2v.wv.key_to_index
print("The total number of words are : ",len(vocab))

The total number of words are :  233702


In [ ]:
vocab=list(vocab.keys())

In [ ]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v.wv.get_vector(word)
print(len(word_vec_dict))

233702


In [ ]:
# maxi=-1
# for i,rev in enumerate(df['text']):
#     tokens=rev.split()
#     if(len(tokens)>maxi):
#         maxi=len(tokens)
# print(maxi)

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(df['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['text'])

In [ ]:
max_rev_len=700  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=100

In [ ]:
from keras.preprocessing.sequence import pad_sequences
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape 

(46678, 700)

In [ ]:
pad_rev

array([[4729,   10,  226, ...,    0,    0,    0],
       [   1, 3335,   95, ...,   53,  439,   58],
       [ 490,   54, 2801, ...,    0,    0,    0],
       ...,
       [ 354,  328,  376, ...,  120,  260,   55],
       [2610,  298,  692, ...,    0,    0,    0],
       [  39, 1143, 7507, ...,    0,    0,    0]], dtype=int32)

In [ ]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix.shape

(93363, 100)

In [ ]:
import keras
import keras.utils
from keras import utils as np_utils
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
sklearn.model_selection.train_test_split

<function sklearn.model_selection._split.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)>

In [ ]:
# !pip install np_utils
from keras.utils.np_utils import to_categorical

In [ ]:
# Y=keras.utils.to_categorical(df['label'])
# Y.shape
from keras import utils as np_utils
Y=keras.utils.np_utils.to_categorical(df['label'])  # one hot target as required by NN.

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(pad_rev, Y, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) # 0.9 x 0.8 = 0.2


In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(Bidirectional(LSTM(128)))
model.add(Flatten())
model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
model.add(Dropout(0.2))

#model.add(Dense(1024,activation='relu'))
# model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
# model.add(Dropout(0.20))
# model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
#model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
# model.add(Dense(1,activation='relu'))
# model.add(Dense(16,activation='relu'))
# model.add(Dropout(0.20))
model.add(Dense(18,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 700, 100)          9336300   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 18)                1170      
                                                        

In [ ]:
from tensorflow import keras
from keras import optimizers

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
epochs=30
batch_size=64

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_val,y_val),callbacks=[es, mc])

Epoch 1/30
526/526 [==============================] - ETA: 0s - loss: 1.9188 - accuracy: 0.5825
Epoch 1: val_loss improved from inf to 0.99512, saving model to best_model.h5
526/526 [==============================] - 70s 116ms/step - loss: 1.9188 - accuracy: 0.5825 - val_loss: 0.9951 - val_accuracy: 0.8054
Epoch 2/30
526/526 [==============================] - ETA: 0s - loss: 0.8772 - accuracy: 0.8132
Epoch 2: val_loss improved from 0.99512 to 0.73099, saving model to best_model.h5
526/526 [==============================] - 59s 111ms/step - loss: 0.8772 - accuracy: 0.8132 - val_loss: 0.7310 - val_accuracy: 0.8327
Epoch 3/30
526/526 [==============================] - ETA: 0s - loss: 0.6664 - accuracy: 0.8477
Epoch 3: val_loss improved from 0.73099 to 0.59168, saving model to best_model.h5
526/526 [==============================] - 60s 114ms/step - loss: 0.6664 - accuracy: 0.8477 - val_loss: 0.5917 - val_accuracy: 0.8563
Epoch 4/30
526/526 [==============================] - ETA: 0s - loss

In [ ]:
modelx = keras.models.load_model('best_model.h5')

In [ ]:
y_pred_one_hot_encoded = (modelx.predict(X_test)> 0.5).astype("float")
#y_pred_test = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))


In [ ]:
print(classification_report(y_test, y_pred_one_hot_encoded))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       268
           1       0.94      0.92      0.93       267
           2       0.87      0.87      0.87       270
           3       0.87      0.82      0.85       228
           4       0.92      0.84      0.88       270
           5       0.89      0.89      0.89       252
           6       0.92      0.90      0.91       250
           7       0.97      0.94      0.95       270
           8       0.88      0.67      0.76       249
           9       0.95      0.87      0.91       252
          10       0.97      0.90      0.94       258
          11       0.98      0.98      0.98       264
          12       0.90      0.86      0.88       241
          13       0.94      0.94      0.94       277
          14       0.96      0.98      0.97       262
          15       0.97      0.94      0.96       247
          16       0.94      0.87      0.90       290
          17       0.98    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
modelx.evaluate(X_test, y_test, batch_size=64)

73/73 [==============================] - 4s 36ms/step - loss: 0.3338 - accuracy: 0.9113


[0.3337896466255188, 0.9113110303878784]

In [ ]:

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [ ]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

In [ ]:
lst_stopword = []
with open('/content/drive/MyDrive/KhanhMD/Data/vietnamese-stopwords.txt','r') as f:
    for line in f:
        for word in line.split():
           lst_stopword.append(word)  
    
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in lst_stopword:
            words.append(word)
    return ' '.join(words)

In [ ]:
document = 'Hãng hàng không VietJet của nữ tỷ phú Nguyễn Thị Phương Thảo có kế hoạch huy động gần nửa tỷ USD để mở rộng hoạt động khi du lịch sôi động trở lại và nhiều hãng hàng không trong nước và quốc tế còn đang khó khăn. CTCP Hàng không Vietjet (VJC) của nữ chủ tịch Nguyễn Thị Phương Thảo vừa công bố nghị quyết của Hội đồng quản trị về phương án phát hành cổ phiếu riêng lẻ 2022 nhằm mở rộng đầu tư, mua thuê máy bay, động cơ, trang thiết bị, bổ sung vốn lưu động,... khi thị trường hàng không sôi động trở lại. Theo đó, VietJet sẽ chào bán riêng lẻ 34,8 triệu cổ phiếu cho 100 nhà đầu tư chứng khoán chuyên nghiệp với giá bán là 135.000 đồng/cp để thu về gần 4.700 tỷ đồng trong năm 2022-2023 sau khi được Ủy ban Chứng khoán Nhà nước chấp thuận. Tiền huy động sẽ được giải ngân ngay trong quý IV/2022 cho tới quý I/2023, trong đó hơn 2.300 tỷ bổ sung thanh khoản và vốn lưu động; hơn 1.100 tỷ đồng cho thuê, mua máy bay; hơn 1.200 tỷ đồng cho thuê, mua động cơ, trang thiết bị,... Nghị quyết HĐQT VietJet. Trước đó, đại hội cổ đông VietJet 2022 đã thông qua kế hoạch phát hành trái phiếu chuyển đổi với tổng giá trị mệnh giá tối đa 6.960 tỷ đồng, hoặc tương đương 300 triệu USD với kỳ hạn 3-5 năm. Tiền thu về cũng sẽ được sử dụng để đầu tư, thuê, mua động cơ và bổ sung nguồn thanh khoản, vốn lưu động hoạt động kinh doanh. Cũng tại ĐHCĐ 2022, cổ đông VietJet đã thông qua phương án phát hành hơn 108,3 triệu cổ phiếu để trả cổ tức cho cổ đông hiện hữu (theo tỷ lệ 10:2) nhằm tăng vốn điều lệ. Nguồn vốn thực hiện lấy từ lợi nhuận sau thuế chưa phân phối đến cuối 2021. Như vậy, tổng cộng với 2 phương án đầu, VietJet sẽ huy động thêm khoảng 11,4 nghìn tỷ đồng (tương đương gần 500 triệu USD). Vốn điều lệ cũng tăng mạnh nhờ hai phương án này và phương án trả cổ tức bằng cổ phiếu. Bà Nguyễn Thị Phương Thảo. Ngành hàng không gần đây ghi nhận kết quả kinh doanh sáng dần với lợi nhuận tiếp tục được duy trì, hoặc đã bớt lỗ. Tuy nhiên, doanh nghiệp vẫn đối mặt với khó khăn do giá nhiên liệu cao và đường quốc tế chưa hồi phục hoàn toàn. Trong quý II/2022, CTCP Hàng không Vietjet (VJC) của nữ tỷ phú Nguyễn Thị Phương Thảo ghi nhận lợi nhuận sau thuế đạt hơn 181 tỷ đồng, nâng lũy kế 6 tháng đầu năm lên gần 426 tỷ đồng. Doanh thu hợp nhất quý II/2022 của VietJet đạt gần 11,6 nghìn tỷ đồng, lũy kế đạt hơn 16,1 nghìn tỷ đồng, tăng mạnh gấp vài lần so với cùng kỳ năm trước và đã cao hơn so với cùng kỳ năm 2019 thời kỳ trước đại dịch Covid-19. Kết quả kinh doanh quý II cũng như 6 tháng đầu năm 2022 của VietJet là khá ấn tượng và nó phản ánh sự hồi phục về nhu cầu đi lại tăng cao sau đại dịch. Theo VietJet, doanh nghiệp đã tăng tần suất khai thác trên nhiều chặng bay thực hiện 52,5 nghìn chuyến bay và vận chuyển 9 triệu lượt hành khách trong nửa đầu  năm 2022, tăng tương ứng gấp rưỡi và gần gấp đôi so với cùng kỳ năm trước. Con số này cũng đã cao hơn so với giai đoạn trước dịch 2019. Nghị quyết ĐHCD 2022 của Viet Jet. Mặc dù ngành hàng không đón những tín hiệu hồi phục hậu đại dịch nhưng khó khăn đối với các doanh nghiệp trong ngành còn nhiều. Giá giá nhiên liệu tăng mạnh và ở mức cao trong thời gian gần đây và đường quốc tế chưa hồi phục hoàn toàn. Tổng công ty Hàng không Việt Nam - Vietnam Airlines (HVN) vừa ghi nhận quý II/2022 tiếp tục thua lỗ gần 2.600 tỷ đồng dù doanh thu tăng gấp gần 3 so với cùng kỳ. Đây là quý lỗ thứ 10 liên tiếp của doanh nghiệp này. Thị trường hàng không nội địa phục hồi nhanh chóng. Tuy nhiên, việc giá nhiên liệu tăng nhanh và mạnh khiến hãng hàng không quốc gia ghi nhận tình trạng kinh doanh dưới giá vốn. Lãi suất có xu hướng tăng cũng ảnh hưởng, khiến chi phí tài chính của doanh nghiệp tăng lên. Mặc dù vậy, mức thua lỗ của Vietnam Airlines không còn nặng nề như khoản lợi nhuận âm hơn 4.500 tỷ đồng trong quý II/2021. Tính trong 6 tháng đầu năm, hãng lỗ hơn 5.250 tỷ đồng, ít hơn gần 3.400 tỷ đồng so với cùng kỳ năm trước. Tuy nhiên, Vietnam Airlines đang ở tình thế ngàn cân treo sợi tóc và chờ thêm một lần giải cứu. Hãng hàng không quốc gia nguy cơ bị hủy niêm yết bắt buộc và khả năng hoạt động liên tục của doanh nghiệp sẽ phụ thuộc sự hỗ trợ tài chính của Chính phủ, việc được gia hạn thanh toán các khoản vay từ ngân hàng… Theo Công ty TNHH Deloitte Việt Nam, khả năng hoạt động liên tục của Vietnam Airlines hiện phụ thuộc sự hỗ trợ tài chính của Chính phủ, việc được gia hạn thanh toán các khoản vay từ ngân hàng, các khoản phải trả cho nhà cung cấp, bên cho thuê. Theo Deloitte, tới cuối quý II/2022, khoản nợ ngắn hạn của hãng hàng không quốc gia Việt Nam đã vượt quá tài sản ngắn hạn. Cụ thể, tới cuối quý II/2022, Vietnam Airlines ghi nhận vốn chủ sở hữu âm gần 4.900 tỷ đồng. Nợ ngắn hạn ở mức rất lớn ở mức gần 52,7 nghìn tỷ đồng, vượt xa so với tài sản ngắn hạn, vốn chỉ ở mức 16,2 nghìn tỷ đồng. Các khoản phải trả quá hạn của Vietnam Airlines là gần 10,5 nghìn tỷ đồng. Sở Giao dịch Chứng khoán TP HCM (HoSE) cũng vừa đưa ra lưu ý khả năng hủy niêm yết bắt buộc cổ phiếu HVN của Vietnam Airlines (HVN). HVN đã vi phạm cả 3 yếu tố mà theo quy định sẽ bị hủy niêm yết bắt buộc. Vietnam Airlines ghi nhận lỗ lũy kế lên tới trên 28,9 nghìn tỷ đồng (tương đương 1,2 tỷ USD).'

document = text_preprocess(document)
document = remove_stopwords(document)
document = [document]
print(document)
#tok.fit_on_texts(df['text'])
encd = tok.texts_to_sequences(document)
encd

['hãng hàng_không vietjet nữ tỷ_phú nguyễn_thị_phương_thảo kế_hoạch huy_động nửa tỷ usd mở_rộng hoạt_động du_lịch sôi_động trở_lại hãng hàng_không quốc_tế khó_khăn _ctcp hàng_không vietjet vjc nữ chủ_tịch nguyễn_thị_phương_thảo công_bố nghị_quyết hội_đồng_quản_trị phương_án phát_hành cổ_phiếu riêng_lẻ 2022 mở_rộng đầu_tư mua thuê máy_bay động_cơ trang_thiết_bị bổ_sung lưu_động thị_trường hàng_không sôi_động trở_lại vietjet chào riêng_lẻ 34 8 triệu cổ_phiếu 100 đầu_tư chứng_khoán chuyên_nghiệp giá_bán 135 000 cp thu 4 700 tỷ 2022 2023 ủy_ban chứng_khoán nhà_nước chấp_thuận _tiền huy_động giải_ngân quý iv 2022 quý i 2023 2 300 tỷ bổ_sung khoản lưu_động 1 100 tỷ thuê mua máy_bay 1 200 tỷ thuê mua động_cơ trang_thiết_bị nghị_quyết hđqt vietjet đại_hội cổ_đông vietjet 2022 thông_qua kế_hoạch phát_hành trái_phiếu chuyển_đổi tổng giá_trị mệnh_giá tối_đa 6 960 tỷ tương_đương 300 triệu usd kỳ_hạn 3 5 _tiền thu sử_dụng đầu_tư thuê mua động_cơ bổ_sung khoản lưu_động hoạt_động kinh_doanh _cũng đhc

[[554,
  56,
  10,
  3804,
  163,
  374,
  652,
  249,
  160,
  247,
  805,
  329,
  742,
  793,
  67,
  1075,
  374,
  509,
  520,
  685,
  369,
  67,
  220,
  259,
  1261,
  67,
  182,
  45,
  554,
  56,
  10,
  77,
  199,
  304,
  676,
  4565,
  56,
  10,
  3804,
  15748,
  163,
  180,
  750,
  249,
  160,
  247,
  805,
  17,
  360,
  697,
  306,
  134,
  41,
  599,
  348,
  247,
  325,
  139,
  113,
  415,
  1162,
  438,
  1405,
  7373,
  520,
  685,
  42,
  263,
  289,
  1029,
  217,
  421,
  67,
  110,
  128,
  266,
  44,
  948,
  1105,
  476,
  67,
  160,
  68,
  56,
  10,
  1261,
  67,
  182,
  45,
  3804,
  1265,
  438,
  1405,
  2206,
  321,
  165,
  415,
  1162,
  709,
  42,
  263,
  467,
  1969,
  385,
  187,
  79,
  206,
  3398,
  243,
  2690,
  213,
  162,
  1866,
  374,
  7373,
  12134,
  1166,
  405,
  467,
  1969,
  49,
  71,
  867,
  791,
  168,
  793,
  67,
  130,
  675,
  798,
  4075,
  7373,
  798,
  1480,
  12134,
  73,
  1250,
  374,
  948,
  1105,
  888,
  476,


In [ ]:
pad= pad_sequences(encd, maxlen=max_rev_len, padding='post')

In [ ]:
predict_x = modelx.predict(pad)
predict_x

array([[3.7167070e-04, 6.4730921e-05, 5.4739282e-04, 3.0696654e-05,
        9.8002028e-01, 6.0456945e-04, 1.1575404e-05, 4.5371130e-03,
        3.6345553e-04, 5.0691166e-04, 1.2763863e-03, 6.1317123e-06,
        1.1211545e-02, 5.1628640e-06, 3.4729627e-04, 5.7554535e-05,
        1.0650909e-06, 3.6420028e-05]], dtype=float32)

In [ ]:
predict_onehot = (modelx.predict(pad)> 0.5).astype("float")
predict_onehot

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [ ]:
make_predict = predict_onehot.argmax(axis=1)
label_encoder.inverse_transform(make_predict)

array(['__label__kinh_doanh'], dtype='<U19')

In [ ]:
for i in range(18):
  print(label_encoder.inverse_transform(np.array([i])))

['__label__công_nghệ']
['__label__du_lịch']
['__label__giáo_dục']
['__label__giải_trí']
['__label__kinh_doanh']
['__label__nhịp_sống']
['__label__phim_ảnh']
['__label__pháp_luật']
['__label__sống_trẻ']
['__label__sức_khỏe']
['__label__thế_giới']
['__label__thể_thao']
['__label__thời_sự']
['__label__thời_trang']
['__label__xe_360']
['__label__xuất_bản']
['__label__âm_nhạc']
['__label__ẩm_thực']
